In [61]:
import tiktoken

In [62]:
enc = tiktoken.get_encoding("cl100k_base")

In [64]:
enc.encode("You are a Finnish teacher.").__len__() + 4 # 4 extra tokens for llama: start_header, end_header, eos, 2 newlines (part of prompt format)

10

In [58]:
from openai import OpenAI

client = OpenAI(
    base_url = "http://dh-dgxh100-2.hpc.msoe.edu:8000/v1",
    api_key = "not_used"
)

out = client.chat.completions.create(
    model="meta/llama-3.1-70b-instruct",
    messages=[
        { "role": "system", "content": "You are a Finnish teacher." },
        {
            "role": "user",
            "content": "What is the most obvious mistake in this sentence (if it has one) \"Tämä on minun koirani\"? Respond in JSON {'correct_sentence': str, 'breif explanation': str|'NA'}.",
        },
    ],
    max_tokens=1024,
    stream=True,
    temperature=0,
    response_format={'type': 'json_object'}
)

for t in out:
    tok = t.choices[0].delta.content
    if not tok: continue
    print(tok, end='')

{"correct_sentence": "Tämä on minun koirani", "breif explanation": "NA"}

---

In [2]:
import llm_core.llm as L
import re

In [35]:
input_toks = L.LLM_GLOBAL_INSTANCE.tokenize(
'''
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a Finnish language teacher.<|eot_id|><|start_header_id|>user<|end_header_id|>

What are the mistakes in "Tämä on minun koira"? Give me: [correct version] [breif list of mistakes].<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''.strip().encode()+b'\n\n', special=True, add_bos=False
)

In [32]:
L.LLM_GLOBAL_INSTANCE.token_bos()

128000

In [38]:
L.LLM_GLOBAL_INSTANCE.tokenize(b'\n\n')

[128000, 271]

In [56]:
L.LLM_GLOBAL_INSTANCE.tokenize(b'assistant', special=True, add_bos=False)

[78191]

In [53]:
L.LLM_GLOBAL_INSTANCE.token_eos()

128009

In [75]:
prompt = [L.LLM_GLOBAL_INSTANCE.token_bos()]

newlines = L.LLM_GLOBAL_INSTANCE.tokenize(b'\n\n', special=True, add_bos=False)[0]
start_header = L.LLM_GLOBAL_INSTANCE.tokenize(b'<|start_header_id|>', special=True, add_bos=False)[0]
end_header = L.LLM_GLOBAL_INSTANCE.tokenize(b'<|end_header_id|>', special=True, add_bos=False)[0]
eos = L.LLM_GLOBAL_INSTANCE.token_eos()

role = {
    name: L.LLM_GLOBAL_INSTANCE.tokenize(name.encode(), special=True, add_bos=False)[0]
    for name in ['system', 'user', 'assistant']
}

for msg in llm._hist:
    msg_content = msg.content
    is_last = msg == llm._hist[-1]
    has_fmt = not not msg.response_format
    # if has_fmt and is_last:
    #     msg_content += f'\n\n\n{response_fmt.pretty_response_format(msg.response_format)}'
    # if has_fmt and not is_last:
    #     msg_content += '\n\n\nRespond in JSON.'

    tokenized_message = L.LLM_GLOBAL_INSTANCE.tokenize(msg_content.encode(), special=False, add_bos=False)

    prompt += [start_header, role[msg.role], end_header, newlines, *tokenized_message, eos]

    if is_last and msg.role == 'user':
        prompt += [start_header, role['assistant'], end_header, newlines]

In [79]:
L.LLM_GLOBAL_INSTANCE(
    prompt,
    max_tokens=20
)

Llama.generate: 8 prefix-match hit, remaining 27 prompt tokens to eval
llama_perf_context_print:        load time =     586.73 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    27 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    19 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    7109.11 ms /    46 tokens


{'id': 'cmpl-fedc1d3d-8d85-4935-8dbf-cc8cb4e49847',
 'object': 'text_completion',
 'created': 1742430273,
 'model': '/data/ai_club/llms/Llama-3.3-70B-Instruct-Q6_K_L-00001-of-00002.gguf',
 'choices': [{'text': 'Here are a few fundamental mathematical constants:\n\n1. **Pi (π)**: approximately 3.',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 35, 'completion_tokens': 20, 'total_tokens': 55}}

In [36]:
L.LLM_GLOBAL_INSTANCE(input_toks, max_tokens=20, temperature=0)

Llama.generate: 50 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     586.73 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    20 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    7320.70 ms /    21 tokens


{'id': 'cmpl-660fdb24-90fa-49d4-9c90-2b78d9d006db',
 'object': 'text_completion',
 'created': 1742429438,
 'model': '/data/ai_club/llms/Llama-3.3-70B-Instruct-Q6_K_L-00001-of-00002.gguf',
 'choices': [{'text': 'Tämä on minun koirani \n* "koira" should be "koir',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 51, 'completion_tokens': 20, 'total_tokens': 71}}

In [3]:
llm = L.LLM('You are a Finnish teacher')

Initializing Global LLM Instance


In [7]:
out = llm('Wazzup!!!!!', response_format='stream')

In [11]:
for t in out:
    print(t, end='')

 hei! Mitä kuuluu? (Hello! How are you?) Let's try to speak some Finnish, shall we? What does "Wazzup" mean to you, and how can we translate it into Finnish? Maybe we can say "Mitä häätä" or "Moi moi"?

In [12]:
llm = L.LLM('You are a statistic and fact sharing bot.')

s = llm(
    'Hello! Give me a few mathematial constants?',
    response_format={
        'constants': [{'name': str, 'value': float}, ...]
    },
    max_tokens=1000,
    temperature=0,
    verbose=False
)

# for t in s:
#     print(t, end='')

print(s)

AssertionError: 

In [12]:
llm = L.LLM('You are a statistic and fact sharing bot.')

s = llm(
    'Hello! What are some Wisconsin cities?',
    response_format={'country': str, 'cities': [{'city_name': str, 'population': int}, 10]},
    max_tokens=1000,
    temperature=0,
    verbose=True
)

# for t in s:
#     print(t, end='')

print(s)

Respond in JSON with this format, writing strings with double quotes and numbers with JS notation (commas end rather than separate digits): {"country": str, "cities": [{"city_name": str, "population": int}, 10 total...]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Generating {'country': <class 'str'>, 'cities': [{'city_name': <class 'str'>, 'population': <class 'int'>}, 10]}
Generating <class 'str'>


/data/ai_club/team_3_2024-25/team3-env-py312-glibc/lib/python3.12/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


Generated: "United States"
Generating [{'city_name': <class 'str'>, 'population': <class 'int'>}, 10]
Generating {'city_name': <class 'str'>, 'population': <class 'int'>}
Generating <class 'str'>
Generated: "Milwaukee"
Generating <class 'int'>
Generated: 595351
Generated: {"city_name": "Milwaukee", "population": 595351}
Generating {'city_name': <class 'str'>, 'population': <class 'int'>}
Generating <class 'str'>
Generated: "Madison"
Generating <class 'int'>
Generated: 255214
Generated: {"city_name": "Madison", "population": 255214}
Generating {'city_name': <class 'str'>, 'population': <class 'int'>}
Generating <class 'str'>
Generated: "Green Bay"
Generating <class 'int'>
Generated: 104779
Generated: {"city_name": "Green Bay", "population": 104779}
Generating {'city_name': <class 'str'>, 'population': <class 'int'>}
Generating <class 'str'>
Generated: "Kenosha"
Generating <class 'int'>
Generated: 99218
Generated: {"city_name": "Kenosha", "population": 99218}
Generating {'city_name': <cl

In [ ]:
s['cities'][2]['city_name']

In [ ]:
llm._hist

In [ ]:
L.LLM.detokenize(llm._hist_to_prompt())

In [ ]:
import json
import re

class Repr:
    def __init__(self, s): self.s = s
    def __str__(self): return self.s
    def __repr__(self): return self.s

def pretty_response_format(response_format):
    if response_format in [str, int, float]:
        return Repr(response_format.__name__)
    elif type(response_format) == dict:
        return {
            Repr(f'"{k}"'): pretty_response_format(v)
            for k,v in response_format.items()
        }
    elif type(response_format) == list:
        sub, etc = response_format

        if etc == ...:
            return [pretty_response_format(sub), Repr('...')]
        elif type(etc) is int:
            return [pretty_response_format(sub), Repr(f'{etc} total...')]

        else:
            raise Exception(f'List size must be int or `...`, but found: {etc}')

    else:
        raise Exception(f'Unsupported value type: {v}')

# fmt = {'country': str, 'capital': str, 'pop': int}
# fmt = {'country': str, 'capital': str, 'pop': int, 'local_names': [str, ...]}
# fmt = {'country': str, 'capital': str, 'pop': int, 'nearest_country': {'name': str, 'pop': int}}
# fmt = {'country': str, 'capital': str, 'pop': int, 'nearest_cities': [{'name': str, 'pop': int}, 3]}
fmt = str

fmt_prompt = pretty_response_format(fmt)

# print(str(fmt_prompt))
# print(f'{fmt_prompt}')
# assert 0

def gen_response_formated_rec(response_format, prompt_ref, temperature=0, max_tokens=1000):
    if response_format == str:
        prompt_ref[0] += '"'
        s = L.LLM_GLOBAL_INSTANCE(
            prompt_ref[0],
            max_tokens=max_tokens, temperature=temperature,
            stream=True,
            # grammar=str_grammar
        )
        for t in s:
            t = t['choices'][0]['text']
            str_end = re.search(r'(^|[^\\])(")', t)
            if str_end:
                prompt_ref[0] += t[:str_end.span(2)[0]]
                break
            prompt_ref[0] += t
        prompt_ref[0] += '"'

    elif response_format in [float, int]:
        s = L.LLM_GLOBAL_INSTANCE(
            prompt_ref[0],
            max_tokens=max_tokens, temperature=temperature,
            stream=True,
            # grammar=num_grammar
        )
        for t in s:
            t = t['choices'][0]['text']
            str_end = re.search(r'[,}]', t)
            if str_end:
                prompt_ref[0] += t[:str_end.span()[0]]
                break
            prompt_ref[0] += t

    elif type(response_format) == dict:
        prompt_ref[0] += '{'
        first = True
        for k,v in response_format.items():
            if not first: prompt_ref[0] += ', '
            first = False
            prompt_ref[0] += f'"{k}": '
            gen_response_formated_rec(v, prompt_ref, temperature, max_tokens)
        prompt_ref[0] += '}'

    elif type(response_format) == list:
        sub_type, etc = response_format

        i = None
        if type(etc) is int:
            i = etc
        else:
            assert etc == ...

        prompt_ref[0] += '['
        while True:
            if not i is None:
                if i == 0: break
                i -= 1

            gen_response_formated_rec(sub_type, prompt_ref, temperature, max_tokens)
            next_tok = L.LLM_GLOBAL_INSTANCE(
                prompt_ref[0],
                max_tokens=1, temperature=0,
                stream=False,
            )['choices'][0]['text'].strip()
            
            if ']' in next_tok:
                break
            elif ',' in next_tok:
                prompt_ref[0] += ', '
                continue
            else:
                raise Exception(f'Theoretically unreachable err: invalid list next tok: {next_tok}')

        prompt_ref[0] += ']'

def gen_response_formated(response_format, prompt, temperature=0, max_tokens=1000):
    json_start = len(prompt)
    prompt_ref = [prompt]
    gen_response_formated_rec(fmt, prompt_ref, temperature, max_tokens)
    return prompt_ref[0][json_start:]

prompt = f'[INST] What is one capital city in Europe? Respond in JSON with this format ending strings with double quotes and numbers with JS notation (commas end rather than separate digits): {fmt_prompt} [/INST] '
print(prompt)

resp = gen_response_formated(fmt, prompt)

print(resp)

# assert 0

# json_start = len(prompt)
# prompt += '{'


# first = True
# for k,v in fmt.items():
#     if not first: prompt += ', '
#     first = False
#     prompt += f'"{k}": '
#     if v == str:
#         prompt += '"'
#         s = L.LLM_GLOBAL_INSTANCE(
#             prompt,
#             max_tokens=1000, temperature=0,
#             stream=True,
#             grammar=str_grammar
#         )
#         for t in s:
#             t = t['choices'][0]['text']
#             str_end = re.search(r'(^|[^\\])(")', t)
#             if str_end:
#                 prompt += t[:str_end.span(2)[0]]
#                 break
#             prompt += t
#         prompt += '"'
#     elif v in [int, float]:
#         s = L.LLM_GLOBAL_INSTANCE(
#             prompt,
#             max_tokens=1000, temperature=0,
#             stream=True,
#             grammar=num_grammar
#         )
#         for t in s:
#             t = t['choices'][0]['text']
#             str_end = re.search(r'[,}]', t)
#             if str_end:
#                 prompt += t[:str_end.span()[0]]
#                 break
#             prompt += t
#     else:
#         raise Exception(f"Unsupported value type: {v}")
# prompt += '}'

# resp = json.loads(prompt[json_start:])

# print(resp)

In [ ]:
try:
    json.loads('owegboieqob')
except json.JSONDecodeError as e:
    raise Exception(f'Cannot parse this json: 23223542')

In [ ]:
llm = L.LLM('You are a Finnish language teacher teaching Finnish to me through conversation. You must give accurate information.')

In [ ]:
s = llm('Oletko koira? respond in English.', response_format='stream', max_tokens=1000, temperature=0.1)
for t in s:
    print(t, end='')

In [ ]:
s = llm('Why did you start your response with an extra space????? AHHHH!!', response_format='stream', max_tokens=1000, temperature=0)
for t in s:
    print(t, end='')

In [ ]:
# llm._hist_to_prompt()
L.LLM.detokenize(llm._hist_to_prompt())
# print(L.LLM.detokenize(llm._hist_to_prompt()))

In [ ]:
llm._hist

In [ ]:
ts = L.LLM_GLOBAL_INSTANCE.tokenize(b'[INST] You are a Finnish language teacher strictly teaching standard Finnish. You must give accurate information. [/INST] Understood.', add_bos=False)
ts = [1, *ts, 2]
ts += L.LLM_GLOBAL_INSTANCE.tokenize(' [INST] What does "mitä on sinun nimi" mean? 1) What is another way to say it? 2) Why is the original version NOT grammatically correct? Tell me in terms of every individual word. [/INST] '.encode('utf-8'), add_bos=False)

In [ ]:

L.LLM.detokenize(ts)

In [ ]:
resp = L.LLM_GLOBAL_INSTANCE(ts, temperature=0, max_tokens=1000, stream=True)
for tok in resp:
    print(tok['choices'][0]['text'], end='')

In [ ]:
resp['choices'][0]['text']

In [ ]:
s = llm('Hello. How are yo?', response_format={'my_name':'...', 'my_response':'...'}, max_tokens=1000)
s

In [ ]:
print(L.LLM.detokenize(llm._hist_to_prompt()))

In [ ]:
$$$  TODO ^ remove the "respond in json" instruction in history

In [ ]:
def logits_processor(prev_tok_ids, next_tok_logits):
    
    # print(prev_tok_ids, next_tok_logits)

    next_tok_logits[995] -= 1000
 
    return next_tok_logits

In [ ]:
# llm = L.LLM('You are a British butler. Start every answer with "you"')
llm = L.LLM('You are a British assistant.')
resp = llm('Hello. What is the capital of Finland?', max_tokens=200, logits_processor=[logits_processor], response_format={'answer_reason':'...', 'answer':'...'})
resp

In [ ]:
print(llm.get_pretty_hist())

In [ ]:
llm.tokenize(' You')

In [ ]:
llm.detokenize([1, 995])